In [ ]:
from transformers import ClapModel, AutoProcessor
import torch
import librosa
import os
import pandas as pd
import numpy as np

In [ ]:
CLAPmodel = "laion/larger_clap_general"
#CLAPmodel = "laion/larger_clap_music_and_speech"
#CLAPmodel = "laion/larger_clap_music"
#CLAPmodel = "laion/clap-htsat-fused"
#CLAPmodel = "laion/clap-htsat-unfused"

In [ ]:
# Load CLAP model + processor
model = ClapModel.from_pretrained(CLAPmodel)
processor = AutoProcessor.from_pretrained(CLAPmodel)

config.json:   0%|          | 0.00/643 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/776M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/776M [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [ ]:
print('total number of CLAP\'s parameters:', sum(p.numel() for p in model.parameters()))

total number of CLAP's parameters: 193913882


In [ ]:
param_size = 0
for param in model.parameters():
    param_size += param.nelement() * param.element_size()
buffer_size = 0
for buffer in model.buffers():
    buffer_size += buffer.nelement() * buffer.element_size()

size_all_mb = (param_size + buffer_size) / 1024**2
print('CLAP model size: {:.3f}MB'.format(size_all_mb))

CLAP model size: 740.294MB


In [ ]:
#This section is to convert the uploaded zip files into folders
import zipfile

# Define the paths to your zip files
zip_files = ["Exp1.zip", "Exp2.zip", "Analysis.zip"]

# Define the directory where you want to extract the files
extract_dir = "/content/" # You can change this if you want to extract elsewhere

# Create the extraction directory if it doesn't exist
os.makedirs(extract_dir, exist_ok=True)

# Extract each zip file
for zip_file in zip_files:
    if os.path.exists(zip_file):
        with zipfile.ZipFile(zip_file, 'r') as zip_ref:
            zip_ref.extractall(extract_dir)
        print(f"Extracted {zip_file} to {extract_dir}")
    else:
        print(f"Erro r: {zip_file} not found.")

Extracted Exp1.zip to /content/
Extracted Exp2.zip to /content/
Extracted Analysis.zip to /content/


# Process audio

In [ ]:
audio_stimuli = []
stimuli_path = "/content/Exp1/Stimuli/"

for file in sorted(os.listdir(stimuli_path)):
    if file.endswith(".wav"):
        wav_path = os.path.join(stimuli_path, file)
        audio, sample_rate = librosa.load(wav_path, sr=48000)
        audio_stimuli.append(audio)

In [ ]:
inputs = processor(audios=audio_stimuli, return_tensors="pt", padding=True, sampling_rate=48000)
audio_embeddings = model.get_audio_features(**inputs)

print(audio_embeddings.shape)

torch.Size([59, 512])


# Process text

In [ ]:
discrete_tags = ["happiness", "sadness", "anger", "tenderness", "fear"]

discrete_captions_perceived = ["I perceive this sound as " + tag for tag in discrete_tags]
print(discrete_captions_perceived)
discrete_captions_induced = ["This sound makes me feel " + tag for tag in discrete_tags]
print(discrete_captions_induced)

dimensional_tags = ["positive", "relaxed", "awake", "like"]

dimensional_captions_perceived = ["I perceive this sound as " + tag for tag in dimensional_tags]
print(dimensional_captions_perceived)
dimensional_captions_induced = ["This sound makes me feel " + tag for tag in dimensional_tags]
print(dimensional_captions_induced)

['I perceive this sound as happiness', 'I perceive this sound as sadness', 'I perceive this sound as anger', 'I perceive this sound as tenderness', 'I perceive this sound as fear']
['This sound makes me feel happiness', 'This sound makes me feel sadness', 'This sound makes me feel anger', 'This sound makes me feel tenderness', 'This sound makes me feel fear']
['I perceive this sound as positive', 'I perceive this sound as relaxed', 'I perceive this sound as awake', 'I perceive this sound as like']
['This sound makes me feel positive', 'This sound makes me feel relaxed', 'This sound makes me feel awake', 'This sound makes me feel like']


In [ ]:
all_tags = discrete_captions_perceived + discrete_captions_induced + dimensional_captions_perceived + dimensional_captions_induced

# NOTE: currently using only dimensional_captions_induced
tag_inputs = processor(text=dimensional_captions_induced, return_tensors="pt", padding=True)
tag_embeds = model.get_text_features(**tag_inputs)

# Generate outputs

In [ ]:
sims = torch.matmul(audio_embeddings, tag_embeds.T)
print(sims.shape)

torch.Size([59, 4])


## Load csv files and extract related columns

In [ ]:
IDim_path = '/content/Exp1/Data/IDim'
IDim_responses = []

all_dfs = []

for file in os.listdir(IDim_path):
    if file.endswith(".csv"):
        df = pd.read_csv(os.path.join(IDim_path, file))
        df.columns = df.columns.str.strip()
        required_columns = ['positive', 'relaxed', 'awake', 'like']
        if all(col in df.columns for col in required_columns):
            all_dfs.append(df)

mean_positive_vector = []
mean_relaxed_vector = []
mean_awake_vector = []
mean_like_vector = []

num_rows = all_dfs[0].shape[0]

for i in range(num_rows):
    current_row_positives = []
    current_row_relaxeds = []
    current_row_awakes = []
    current_row_likes = []

    # For the current row index 'i', collect values from all DataFrames
    for df in all_dfs:
        current_row_positives.append(df.iloc[i]['positive'])
        current_row_relaxeds.append(df.iloc[i]['relaxed'])
        current_row_awakes.append(df.iloc[i]['awake'])
        current_row_likes.append(df.iloc[i]['like'])

    # Calculate the mean for the current row across all files, for each column
    mean_positive_vector.append(np.mean(current_row_positives))
    mean_relaxed_vector.append(np.mean(current_row_relaxeds))
    mean_awake_vector.append(np.mean(current_row_awakes))
    mean_like_vector.append(np.mean(current_row_likes))

IDim_responses = {
    'positive_vector': mean_positive_vector,
    'relaxed_vector': mean_relaxed_vector,
    'awake_vector': mean_awake_vector,
    'like_vector': mean_like_vector
}

print("\n--- Mean Vectors (Mean across CSVs for each row position) ---")
print(f"Length of positive_vector: {len(IDim_responses['positive_vector'])}")
print(f"Length of relaxed_vector: {len(IDim_responses['relaxed_vector'])}")
print(f"Length of awake_vector: {len(IDim_responses['awake_vector'])}")
print(f"Length of like_vector: {len(IDim_responses['like_vector'])}")

# Find the min and max values in the current sims tensor
old_min = sims.min()
old_max = sims.max()

# Define the new desired range
new_min = 1.0
new_max = 9.0

# Apply the min-max scaling formula
scaled_sims = ((sims - old_min) / (old_max - old_min)) * (new_max - new_min) + new_min

print(f"Scaled sims shape: {scaled_sims.shape}")
print(f"Scaled sims (first 5 rows):\n{scaled_sims[:10]}\n")
print(f"Scaled sims min value: {scaled_sims.min():.4f}")
print(f"Scaled sims max value: {scaled_sims.max():.4f}\n")

human_ratings_tensor = torch.tensor([
    IDim_responses['positive_vector'],
    IDim_responses['relaxed_vector'],
    IDim_responses['awake_vector'],
    IDim_responses['like_vector']
], dtype=torch.float32).T # Transpose to get shape (59, 3)

print(f"Human ratings tensor shape: {human_ratings_tensor.shape}")
print(f"Human ratings tensor (first 5 rows):\n{human_ratings_tensor[:10]}\n")


--- Mean Vectors (Mean across CSVs for each row position) ---
Length of positive_vector: 59
Length of relaxed_vector: 59
Length of awake_vector: 59
Length of like_vector: 59
Scaled sims shape: torch.Size([59, 4])
Scaled sims (first 5 rows):
tensor([[5.5154, 6.2459, 4.3756, 5.5424],
        [5.0735, 6.2841, 3.4231, 4.8334],
        [4.5980, 6.2105, 4.2557, 4.7714],
        [3.6465, 5.2515, 4.2829, 4.6969],
        [4.3135, 3.9755, 2.7836, 4.8127],
        [3.4054, 3.1892, 2.4026, 2.8802],
        [5.0046, 4.9204, 3.4381, 4.0295],
        [5.6248, 4.9287, 3.4671, 4.2985],
        [5.4228, 5.2849, 3.4730, 3.8950],
        [5.7928, 5.7447, 4.3253, 5.2209]], grad_fn=<SliceBackward0>)

Scaled sims min value: 1.0000
Scaled sims max value: 9.0000

Human ratings tensor shape: torch.Size([59, 4])
Human ratings tensor (first 5 rows):
tensor([[4.4872, 4.8798, 4.5185, 4.6463],
        [4.7682, 5.3057, 4.7414, 5.0326],
        [5.3105, 5.7312, 5.0760, 5.3854],
        [5.2582, 5.2651, 5.4488, 5.249

# Evaluate

In [ ]:
from scipy.stats import pearsonr # For Pearson correlation
print("Results for Dimensional Induced (IDim)")
print("______________________________________________")
# Comparison Method 1: Mean Absolute Error (MAE)
# MAE measures the average magnitude of the errors in a set of predictions, without considering their direction.
mae = torch.mean(torch.abs(scaled_sims - human_ratings_tensor))
print(f"Mean Absolute Error (MAE) between scaled_sims and human_ratings: {mae:.4f}\n")

# Comparison Method 2: Mean Absolute Percentage Error (MAPE)
# MAPE measures the accuracy of a forecasting method in terms of percentage.
# Formula: MAPE = (1/n) * sum(|(Actual - Forecast) / Actual|) * 100%

# Calculate the absolute percentage error for each element
# Since human responses are between 1 and 9, division by zero is not a concern.
absolute_percentage_error = torch.abs((human_ratings_tensor - scaled_sims) / human_ratings_tensor) * 100

# Calculate the mean of these percentage errors
mape = torch.mean(absolute_percentage_error)

print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%\n")

# Comparison Method 3: Root Mean Squared Error (RMSE)
# RMSE measures the square root of the average of the squared differences between predicted and actual values.
# It gives a relatively high weight to large errors.
# Formula: RMSE = sqrt(mean((Actual - Forecast)^2))

# Calculate the squared differences
squared_differences = (human_ratings_tensor - scaled_sims)**2

# Calculate the mean of the squared differences (Mean Squared Error - MSE)
mse = torch.mean(squared_differences)

# Calculate the square root to get RMSE
rmse = torch.sqrt(mse)

print(f"Root Mean Squared Error (RMSE): {rmse:.4f}\n")

# Comparison Method 4: Pearson Correlation Coefficient (per column)
# Pearson correlation measures the linear relationship between two sets of data.
# We'll calculate it for each of the three columns (dimensions).

correlation_positive, _ = pearsonr(scaled_sims[:, 0].detach().numpy(), human_ratings_tensor[:, 0].numpy())
correlation_relaxed, _ = pearsonr(scaled_sims[:, 1].detach().numpy(), human_ratings_tensor[:, 1].numpy())
correlation_awake, _ = pearsonr(scaled_sims[:, 2].detach().numpy(), human_ratings_tensor[:, 2].numpy())
correlation_like, _ = pearsonr(scaled_sims[:, 3].detach().numpy(), human_ratings_tensor[:, 3].numpy())

print("Pearson Correlation Coefficients (between scaled_sims and human_ratings):")
print(f"  Positive Dimension: {correlation_positive:.4f}")
print(f"  Relaxed Dimension:  {correlation_relaxed:.4f}")
print(f"  Awake Dimension:    {correlation_awake:.4f}")
print(f"  Like Dimension:     {correlation_like:.4f}")

average_correlation = (correlation_positive + correlation_relaxed + correlation_awake + correlation_like) / 4
print(f"  Average Correlation: {average_correlation:.4f}")

from sklearn.metrics import r2_score

# R-squared
print("\nR-squared scores:")
r2_valence = r2_score(human_ratings_tensor[:, 0].numpy(), scaled_sims[:, 0].detach().numpy())
print(f"  valence = {r2_valence:.4f}")

r2_tension = r2_score(human_ratings_tensor[:, 1].numpy(), scaled_sims[:, 1].detach().numpy())
print(f"  tension = {r2_tension:.4f}")

r2_energy = r2_score(human_ratings_tensor[:, 2].numpy(), scaled_sims[:, 2].detach().numpy())
print(f"  energy = {r2_energy:.4f}")

r2_like = r2_score(human_ratings_tensor[:, 3].numpy(), scaled_sims[:, 3].detach().numpy())
print(f"  like = {r2_like:.4f}")

r2_scores = [r2_valence, r2_tension, r2_energy, r2_like]

# Calculate the average R²
average_r2 = np.mean(r2_scores)
print(f"\nAverage R-squared across dimensions: {average_r2:.4f}")

Results for Dimensional Induced (IDim)
______________________________________________
Mean Absolute Error (MAE) between scaled_sims and human_ratings: 1.5548

Mean Absolute Percentage Error (MAPE): 29.78%

Root Mean Squared Error (RMSE): 2.0774

Pearson Correlation Coefficients (between scaled_sims and human_ratings):
  Positive Dimension: 0.6344
  Relaxed Dimension:  0.6316
  Awake Dimension:    -0.4555
  Like Dimension:     0.1266
  Average Correlation: 0.2343

R-squared scores:
  valence = -0.2843
  tension = 0.1653
  energy = -31.5805
  like = -0.6736

Average R-squared across dimensions: -8.0932
